In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/My \Drive/Sem \8 \files/
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Sem 8 files
 data   euclidean_distances.json  'Fri Feb 14 09_06_16 2020.json'


In [2]:
!apt-get install -y libspatialindex-c4v5
!pip3.6 install osmnx

import osmnx as ox
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import math

from shapely.geometry import box
from heapq import heappush, heappop
from itertools import count
from rtree import index

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-c4v5 is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [3]:
place_name = "Singapore, Central, Singapore"
G = ox.save_load.load_graphml(filename="Singapore_drive_processed.graphml")

# Stores nodes and edges mapped with speed bands along with other attributes
type(G)

networkx.classes.multidigraph.MultiDiGraph

In [0]:
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
nodes = pd.read_pickle('data/nodes_drive.pkl')

In [0]:
def process_speed_band(df):
    
    # Define Locations
    def truncate(number, digits) -> float:
        stepper = 10.0 ** digits
        return math.trunc(stepper * number) / stepper
    
    location = df['Location'].values
    i = 0
    
    for loc in location:
        x1, y1, x2, y2 = [float(n) for n in loc.split(' ')]
        
        x1 = truncate(x1, 7)
        y1 = truncate(y1, 7)
        x2 = truncate(x2, 7)
        y2 = truncate(y2, 7)
        
        if y1 < y2:
            bottom = y1
            top = y2
        else:
            bottom = y2
            top = y1
            
        if x1 < x2:
            left = x1
            right = x2
        else:
            left = x2
            right = x1
        
        df['Location'].values[i] = (left, bottom, right, top)
        i += 1 
    
    # Process maximum speeds
    i = 0
    for x in df['SpeedBand']:
        if x == 0:
            df['MaximumSpeed'][i] = '50'
        i += 1 

In [7]:
f = open("Fri Feb 14 09_06_16 2020.json", "r").read()
x = json.loads(f)

speed_bands = pd.DataFrame.from_dict(x, orient='columns')
process_speed_band(speed_bands)
# speed_bands.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
def calculate_congestion(G, edges, speed_bands):
    # Define Location    
    location = edges['geometry'].values
    i = 0
    Location = []
    
    for loc in location:        
        x1 = loc.xy[1][-1]
        y1 = loc.xy[0][-1]
        x2 = loc.xy[1][0]
        y2 = loc.xy[0][0]
        
        if y1 < y2:
            bottom = y1
            top = y2
        else:
            bottom = y2
            top = y1
            
        if x1 < x2:
            left = x1
            right = x2
        else:
            left = x2
            right = x1
        
        Location.append((left, bottom, right, top))
        i += 1 
        
    edges['Location'] = Location
    
    
    # Define observed_speed using speedband dataset 
    
    # key: maximum speed ,value: location
    idx = index.Index()
    Location = speed_bands['Location']
    Speed = speed_bands['MaximumSpeed'].astype(float)

    for speed, loc in zip(Speed, Location):
        idx.insert(int(speed), loc)
    
    # Find intersections
    observed_speed = []
    Location = edges['Location']
    speeds =  edges['maxspeed']

    for loc, speed in zip(Location, speeds):
      max_speeds = list(idx.intersection(loc))

      if len(max_speeds) == 0: # edge didn't intersect with any speed band
          observed_speed.append(speed)
      else:
          observed_speed.append(sum(max_speeds)/len(max_speeds))

    edges['observed_speed'] = observed_speed
    
    
    # Define BPR heuristic for each road link
    bpr = dict()
    n = len(edges['u'])

    for i in range (n):
        u = edges['u'][i]
        v = edges['v'][i]
        key = edges['key'][i]
        time = float(edges['travel_time'][i])
        flow = float(edges['observed_speed'][i])
        capacity = float(edges['maxspeed'][i])

        bpr[(u,v,key)] = time * (1 + 0.15*(flow/capacity)**4)
        
    nx.set_edge_attributes(G, bpr, 'BPR')


    # Define capacity as a random number between 835-905
    import random
    capacity  = dict()

    for i in range(n):
        x = random.randint(835, 905)
        u = edges['u'][i]
        v = edges['v'][i]
        key = edges['key'][i]
        capacity[(u,v,key)] = x

    nx.set_edge_attributes(G, capacity, 'capacity')

In [0]:
calculate_congestion(G, edges, speed_bands)
# edges.head()

In [11]:
len(nodes['osmid'].unique())

23219

In [0]:
paths = pd.DataFrame(np.zeros(shape=(23219,23219)), columns=nodes['osmid'].unique(), index=nodes['osmid'].unique())

In [14]:
for n in nodes['osmid']:
  for m in nodes['osmid']:
    try:
      d = nx.shortest_path_length(G, source=n, target=m, weight='length')
    except nx.NetworkXNoPath:
      d = float('inf')
  
    paths[n][m] = d
    print(paths[n][m])

Streaming output truncated to the last 5000 lines.
2329.2619999999997
19240.686000000005
2529.065
2508.013
18742.012000000006
18753.484000000008
20777.447
18140.368000000002
17436.242000000002
20947.025999999998
18109.058
17329.280000000002
17408.499000000003
21832.69000000001
15427.483999999995
17452.060000000005
15372.955000000002
17430.626000000004
15281.017000000002
15190.585000000003
17542.343000000004
14958.688999999998
17501.815000000002
14971.212000000001
14837.661
17640.048000000003
22203.718000000004
17329.015000000003
17394.763000000003
14080.104999999998
17650.602000000006
17847.788
17907.665
17873.082000000006
16577.329000000005
22042.211000000003
16185.544000000005
16179.191000000006
11729.698000000002
16437.338000000003
16896.868000000006
11201.096
18060.464000000007
22031.343000000004
17898.784000000003
17908.696000000004
17872.671000000002
22264.135
18107.014000000003
16265.198999999999
3580.806000000001
17709.701000000005
17742.500000000007
12210.022999999997
18195.35

KeyboardInterrupt: ignored

In [0]:
paths.to_pickle('Shortest_path_lengths.pkl')

In [0]:
df = pd.read_pickle('Shortest_path_lengths.pkl')

In [15]:
paths[1820000257][3874553858]

6457.045999999998